In [1]:
def generateEnvision(Observation=("Items", "Y"), upstream=[("Model", "X"), ("Size", "X")]):
    
    (obs, y) = Observation
    
    r = "/// Smart Init \n"
    
    def addLine(r, text): 
        return r + '\n' + text
    
    def skipLine(r): 
        return r + '\n'
    

        
    r = skipLine(r)
        
    pairs = [(a, b) for idx, a in enumerate(upstream) for b in upstream[idx + 1:]]
        
    
    for (t1, v1), (t2, v2) in pairs:
        r = addLine(r, "////// %s x %s" % (t1, t2))
        crossTable = "%sx%s" % (t1, t2)
        r = addLine(r, "table %s = by[%s.X%s, %s.X%s]" % (crossTable, obs, t1, obs, t2) )
        r = addLine(r, "%s.%s = avg(%s.%s)" % (crossTable, y, obs, y))
        
        
        
        
        r = addLine(r, "%s.%s%s = sum(%s.%s)" % (t1, v1, t2, crossTable, y))
        r = addLine(r, "%s%s%s = sum(%s.%s%s)" % (t1, v1, t2, t1, v1, t2))
        r = addLine(r, "%s.%s%s = %s.%s%s / sqrt(%s%s%s)" % (t1, v1 ,t2, t1, v1,t2, t1, v1, t2))
        t1, t2 = t2, t1
        v1, v2 = v2, v1
        

        r = addLine(r, "%s.%s%s = sum(%s.%s )" % (t1, v1, t2, crossTable, y))
        r = addLine(r, "%s%s%s = sum(%s.%s%s)" % (t1, v1, t2, t1, v1, t2))
        r = addLine(r, "%s.%s%s = %s.%s%s / sqrt(%s%s%s)" % (t1, v1 ,t2, t1, v1,t2, t1, v1, t2))
        t1, t2 = t2, t1
        v1, v2 = v2, v1
        
        r = addLine(r, "delete %s.X%s" % (crossTable, t1))
        r = addLine(r, "delete %s.X%s" % (crossTable, t2))
        r = skipLine(r)

    r = skipLine(r)

    
    
    
    r = addLine(r, "avg%s%s = avg(%s.%s)" % (obs, y, obs, y))
    for (t, v) in upstream:
        otherTables = [t + "." + v + ot for (ot, _) in upstream if t != ot]
        s = " + ".join(otherTables)
        r = addLine(r, "%s.%s = (%s) / %i" % (t, v, s, len(otherTables)))
        
        
        r = addLine(r, "%s%s = sum(%s.%s^2)" % (t, v, t, v))
        r = addLine(r, "%s.%s = %s.%s / sqrt(%s%s) " % (t, v, t, v, t, v))
        r = addLine(r, "avg%s%s = avg(%s.%s)" %(t, v, t, v))
        r = addLine(r, "%s.%s = %s.%s / avg%s%s " % (t, v, t, v, t, v))
        r = addLine(r, "%s.%s = %s.%s * (avg%s%s^(1/%i)) " % (t, v, t, v, obs, y, len(upstream)))
        
    r = skipLine(r)
        
    for (t, v) in upstream:
        
        r = addLine(r, "%s.Max = max(%s.%s)" % (t, obs, y))
        r = addLine(r, "%s.Min = min(%s.%s)" % (t, obs, y))
        r = addLine(r, "%s.%s = min(%s.%s, %s.Max / %i)" % (t, v, t, v, t, len(upstream)))
        r = addLine(r, "%s.%s = max(%s.%s, %s.Min / %i)" % (t, v, t, v, t, len(upstream)))
        

        
    r = addLine(r, "\n/// End Smart Init")
    return r

In [2]:
print(generateEnvision())

/// Smart Init 


////// Model x Size
table ModelxSize = by[Items.XModel, Items.XSize]
ModelxSize.Y = avg(Items.Y)
Model.XSize = sum(ModelxSize.Y)
ModelXSize = sum(Model.XSize)
Model.XSize = Model.XSize / sqrt(ModelXSize)
Size.XModel = sum(ModelxSize.Y )
SizeXModel = sum(Size.XModel)
Size.XModel = Size.XModel / sqrt(SizeXModel)
delete ModelxSize.XModel
delete ModelxSize.XSize


avgItemsY = avg(Items.Y)
Model.X = (Model.XSize) / 1
ModelX = sum(Model.X^2)
Model.X = Model.X / sqrt(ModelX) 
avgModelX = avg(Model.X)
Model.X = Model.X / avgModelX 
Model.X = Model.X * (avgItemsY^(1/2)) 
Size.X = (Size.XModel) / 1
SizeX = sum(Size.X^2)
Size.X = Size.X / sqrt(SizeX) 
avgSizeX = avg(Size.X)
Size.X = Size.X / avgSizeX 
Size.X = Size.X * (avgItemsY^(1/2)) 

Model.Max = max(Items.Y)
Model.Min = min(Items.Y)
Model.X = min(Model.X, Model.Max / 2)
Model.X = max(Model.X, Model.Min / 2)
Size.Max = max(Items.Y)
Size.Min = min(Items.Y)
Size.X = min(Size.X, Size.Max / 2)
Size.X = max(Size.X, Size.Min / 2)


In [3]:
print(generateEnvision(Observation=("Items", "SoldQty"), upstream=[("Model", "Model"), ("Size", "Size"), ("Color", "Color"), ("Store", "Store")]))

/// Smart Init 


////// Model x Size
table ModelxSize = by[Items.XModel, Items.XSize]
ModelxSize.SoldQty = avg(Items.SoldQty)
Model.ModelSize = sum(ModelxSize.SoldQty)
ModelModelSize = sum(Model.ModelSize)
Model.ModelSize = Model.ModelSize / sqrt(ModelModelSize)
Size.SizeModel = sum(ModelxSize.SoldQty )
SizeSizeModel = sum(Size.SizeModel)
Size.SizeModel = Size.SizeModel / sqrt(SizeSizeModel)
delete ModelxSize.XModel
delete ModelxSize.XSize

////// Model x Color
table ModelxColor = by[Items.XModel, Items.XColor]
ModelxColor.SoldQty = avg(Items.SoldQty)
Model.ModelColor = sum(ModelxColor.SoldQty)
ModelModelColor = sum(Model.ModelColor)
Model.ModelColor = Model.ModelColor / sqrt(ModelModelColor)
Color.ColorModel = sum(ModelxColor.SoldQty )
ColorColorModel = sum(Color.ColorModel)
Color.ColorModel = Color.ColorModel / sqrt(ColorColorModel)
delete ModelxColor.XModel
delete ModelxColor.XColor

////// Model x Store
table ModelxStore = by[Items.XModel, Items.XStore]
ModelxStore.SoldQty = avg(I

In [4]:
print(generateEnvision(Observation=("A", "A"), upstream=[("Model", "Model"), ("Size", "Size"), ("Color", "Color")]))

/// Smart Init 


////// Model x Size
table ModelxSize = by[A.XModel, A.XSize]
ModelxSize.A = avg(A.A)
Model.ModelSize = sum(ModelxSize.A)
ModelModelSize = sum(Model.ModelSize)
Model.ModelSize = Model.ModelSize / sqrt(ModelModelSize)
Size.SizeModel = sum(ModelxSize.A )
SizeSizeModel = sum(Size.SizeModel)
Size.SizeModel = Size.SizeModel / sqrt(SizeSizeModel)
delete ModelxSize.XModel
delete ModelxSize.XSize

////// Model x Color
table ModelxColor = by[A.XModel, A.XColor]
ModelxColor.A = avg(A.A)
Model.ModelColor = sum(ModelxColor.A)
ModelModelColor = sum(Model.ModelColor)
Model.ModelColor = Model.ModelColor / sqrt(ModelModelColor)
Color.ColorModel = sum(ModelxColor.A )
ColorColorModel = sum(Color.ColorModel)
Color.ColorModel = Color.ColorModel / sqrt(ColorColorModel)
delete ModelxColor.XModel
delete ModelxColor.XColor

////// Size x Color
table SizexColor = by[A.XSize, A.XColor]
SizexColor.A = avg(A.A)
Size.SizeColor = sum(SizexColor.A)
SizeSizeColor = sum(Size.SizeColor)
Size.SizeColor 

In [10]:
def generateEnvisionAutreIdee(Observation=("Items", "Y"), upstream=[("Model", "X"), ("Size", "X")]):
    
    (obs, y) = Observation
    
    r = "/// Smart Init \n"
    
    def addLine(r, text): 
        return r + '\n' + text
    
    def skipLine(r): 
        return r + '\n'
    

        
    r = skipLine(r)
        
    pairs = [(a, b) for idx, a in enumerate(upstream) for b in upstream[idx + 1:]]
        
    
    for (t1, v1), (t2, v2) in pairs:
        r = addLine(r, "////// %s x %s" % (t1, t2))
        crossTable = "%sx%s" % (t1, t2)
        r = addLine(r, "table %s = by[%s.X%s, %s.X%s]" % (crossTable, obs, t1, obs, t2) )
        r = addLine(r, "%s.%s = avg(%s.%s)" % (crossTable, y, obs, y))
        
        
        r = addLine(r, "%s.%s = -1234" % (crossTable, v1))
        r = addLine(r, "%s.%s = -1234" % (crossTable, v2))
        r = addLine(r, "firstId = whichever(%s.X%s) where %s.%s > 0" % (crossTable, v1, crossTable, y))
        r = addLine(r, "where %s.X%s == firstId" % (crossTable, v1))
        r = addLine(r, "  %s.%s = 1" % (crossTable, v1))
        r = addLine(r, "  %s.%s = %s.%s" % (crossTable, v2, crossTable, y))
        
        r = skipLine(r)

        
        r = addLine(r, "%s.%s%s = max(%s.%s)" % (t1, v1,v2, crossTable, v1))
        r = addLine(r, "%s.%s = %s.%s%s" % (crossTable, v1, t1, v1, v2))
        r = addLine(r, "%s.%s%s = max(%s.%s)" % (t2, v2,v1, crossTable, v2))
        r = addLine(r, "%s.%s = %s.%s%s" % (crossTable, v2, t2, v2, v1))

        r = skipLine(r)

        
        for _ in range(4):
            
            r = addLine(r, "where %s.%s != -1234 and %s.%s == -1234" % (crossTable, v2, crossTable, v1) )
            r = addLine(r, "  %s.%s = %s.%s / %s.%s" % (crossTable, v1, crossTable, y, crossTable, v2))
            # r = addLine(r, "%s.%s%s = max(%s.%s)" % (t1, v1, v2, crossTable, v1))
            # r = addLine(r, "%s.%s%s = max(%s.%s)" % (t2, v2, v1, crossTable, v2))
            
            
            t1, t2 = t2, t1
            v1, v2 = v2, v1
            
        r = skipLine(r)

        for _ in range(2):
            r = addLine(r, "%s.%s%s = whichever(%s.%s)" % (t1, v1, v2, crossTable, v1))
            
            r = addLine(r, "avg%s%s = avg(%s.%s%s)" % (v1, v2, t1, v1, v2))
            r = addLine(r, "%s.%s%s = %s.%s%s / avg%s%s" % (t1, v1, v2, t1, v1, v2, v1, v2))
            t1, t2 = t2, t1
            v1, v2 = v2, v1
            
            
        r = addLine(r, "delete %s.X%s" % (crossTable, t1))
        r = addLine(r, "delete %s.X%s" % (crossTable, t2))
        r = skipLine(r)
            
    r = skipLine(r)

    r = addLine(r, "avg%s%s = avg(%s.%s)" % (obs, y, obs, y))
    for (t, v) in upstream:
        otherTables = [t + "." + v + ov for (ot, ov) in upstream if t != ot]
        s = " + ".join(otherTables)
        r = addLine(r, "%s.%s = (%s) / %i" % (t, v, s, len(otherTables)))     
        r = addLine(r, "norm%s = sum(%s.%s^2)" % (t, t, v))
        r = addLine(r, "%s.%s = (%s.%s) / norm%s" % (t, v, t, v, t))     
        r = addLine(r, "avg%s = avg(%s.%s)" % (t, t, v))
        r = addLine(r, "%s.%s = %s.%s * (avg%s%s^(1/%i)) / (avg%s) " % (t, v, t, v, obs, y, len(upstream), t))
        
    r = skipLine(r)
        
#     for (t, v) in upstream:
        
#         r = addLine(r, "%s.Max = max(%s.%s)" % (t, obs, y))
#         r = addLine(r, "%s.Min = min(%s.%s)" % (t, obs, y))
#         r = addLine(r, "%s.%s = min(%s.%s, %s.Max / %i)" % (t, v, t, v, t, len(upstream)))
#         r = addLine(r, "%s.%s = max(%s.%s, %s.Min / %i)" % (t, v, t, v, t, len(upstream)))


        
    r = addLine(r, "\n/// End Smart Init")
    return r

In [11]:
print(generateEnvisionAutreIdee(Observation=("SKUs", "Y"), upstream=[("Model", "Model"), ("Size", "Size"), ("Color", "Color"), ("Store", "Store")]))

/// Smart Init 


////// Model x Size
table ModelxSize = by[SKUs.XModel, SKUs.XSize]
ModelxSize.Y = avg(SKUs.Y)
ModelxSize.Model = -1234
ModelxSize.Size = -1234
firstId = whichever(ModelxSize.XModel) where ModelxSize.Y > 0
where ModelxSize.XModel == firstId
  ModelxSize.Model = 1
  ModelxSize.Size = ModelxSize.Y

Model.ModelSize = max(ModelxSize.Model)
ModelxSize.Model = Model.ModelSize
Size.SizeModel = max(ModelxSize.Size)
ModelxSize.Size = Size.SizeModel

where ModelxSize.Size != -1234 and ModelxSize.Model == -1234
  ModelxSize.Model = ModelxSize.Y / ModelxSize.Size
where ModelxSize.Model != -1234 and ModelxSize.Size == -1234
  ModelxSize.Size = ModelxSize.Y / ModelxSize.Model
where ModelxSize.Size != -1234 and ModelxSize.Model == -1234
  ModelxSize.Model = ModelxSize.Y / ModelxSize.Size
where ModelxSize.Model != -1234 and ModelxSize.Size == -1234
  ModelxSize.Size = ModelxSize.Y / ModelxSize.Model

Model.ModelSize = whichever(ModelxSize.Model)
avgModelSize = avg(Model.ModelSize)
Mod